  Author: Ankit Kariryaa, University of Bremen
  
  Modified by Xuehui Pi and Qiuqi Luo



In [1]:
from tensorflow.keras.models import load_model
import numpy as np               # numerical array manipulation
import pandas as pd
import geopandas as gps
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import time
from collections import defaultdict
from functools import reduce
from PIL import Image
import rasterio                  # I/O raster data (netcdf, height, geotiff, ...)
import rasterio.warp             # Reproject raster samples
from shapely.geometry import Point, Polygon
from shapely.geometry import mapping, shape
import fiona
import cv2
from tqdm import tqdm
import rasterio.mask
import affine

from core.UNet import UNet
from core.losses import tversky, accuracy, dice_coef, dice_loss, IoU, recall, precision
from core.optimizers import adaDelta, adagrad, adam, nadam
from core.frame_info import FrameInfo
from core.dataset_generator import DataGenerator
from core.visualize import display_images

%matplotlib inline
import matplotlib.pyplot as plt  # plotting tools
import matplotlib.patches as patches
import random
import warnings                  # ignore annoying warnings
warnings.filterwarnings("ignore")
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'

import tensorflow as tf
print(tf.__version__)

2.5.0-rc3


In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto(
    #device_count={"CPU": 64},
    allow_soft_placement=True, 
    log_device_placement=False)
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  2


In [41]:
# Initialize the data related variables used in the notebook 

# For reading the GSW and annotated images generated in the step - 1

base_dir = r'D:\sample250\U-Net_13' 
type_num = 5
image_type = '.png'
NDWI_fn = 'ndwi'
red_fn = 'red'
blue_fn = 'blue'
green_fn = 'green'
swir_fn = 'swir'
annotation_fn = 'annotation'

# For testing, images are divided into sequential patches 
patch_generation_stratergy = 'sequential'
patch_size = (512,512,2) ## Height * Width * (Input or Output) channels：[GSW, ANNOTATION]
BATCH_SIZE = 16 # Model is evaluated in batches; See https://keras.io/models/model/

# # When stratergy == sequential
step_size = (512,512)

# input_shape = (512,512,5)
# input_image_channel = [0,1,2,3,4]
# input_label_channel = [5]

# input_shape = (512,512,2)
# input_image_channel = [0,1]
# input_label_channel = [2]

input_shape = (512,512,1)
input_image_channel = [0]
input_label_channel = [1]

OPTIMIZER_NAME = 'adaDelta'
OPTIMIZER = adaDelta 
# OPTIMIZER=tf.train.experimental.enable_mixed_precision_graph_rewrite(OPTIMIZER)

In [35]:
LOSS = dice_loss
# LOSS=tf.keras.losses.BinaryCrossentropy()
# LOSS_NAME = 'tversky'
LOSS_NAME = 'dice_loss'
# modelToEvaluate = os.path.join(base_dir, r'saved_models\UNet\lakes_20230818-2031_AdaDelta_dice_loss_b5_012345_512.h5')
# modelToEvaluate = os.path.join(base_dir, r'saved_models\UNet\lakes_20230819-0235_AdaDelta_dice_loss_b5_normalized_012345_512.h5')
# modelToEvaluate = os.path.join(base_dir, r'saved_models\UNet\lakes_20230819-0232_AdaDelta_dice_loss_b2_normalized_012_512.h5')
modelToEvaluate = os.path.join(base_dir, r'saved_models\UNet\lakes_20230819-1052_AdaDelta_dice_loss_b1_01_512.h5')

In [36]:
#File path for final report 
timestr = time.strftime("%Y%m%d-%H%M")
chf = input_image_channel + input_label_channel
chs = reduce(lambda a,b: a+str(b),   chf, '')
evaluation_report_path = model_path =  os.path.join(base_dir, 'evaluationreport') 
if not os.path.exists(evaluation_report_path):
    os.makedirs(evaluation_report_path)
evaluation_report_filename = os.path.join(evaluation_report_path,'evaluation_per_pixel{}_{}.csv'.format(timestr,chs))
print(evaluation_report_filename) 

D:\sample250\U-Net_13\evaluationreport\evaluation_per_pixel20230819-1507_012.csv


In [37]:
def readBands(path_to_write,fn):
    img=rasterio.open(os.path.join(path_to_write,fn))
    
#     read_img=img.read()/1000
    
    im=img.read()
    axis=(0, 1)
    read_img=(im - im.mean(axis)) / (im.std(axis) + 1e-8)
    return read_img

def readImgs(path_to_write, fn):
    NDWI_img = rasterio.open(os.path.join(path_to_write, fn))
    read_NDWI_img = NDWI_img.read()/100
    
    comb_img = np.transpose(read_NDWI_img, axes=(1,2,0))
    
    #if use other bands for trainging, use codes as belows:
#     read_red_img =readBands(path_to_write,fn.replace(NDWI_fn ,red_fn))
#     read_green_img =readBands(path_to_write,fn.replace(NDWI_fn ,green_fn))
#     read_blue_img = readBands(path_to_write,fn.replace(NDWI_fn ,blue_fn))
#     read_swir_img = readBands(path_to_write, fn.replace(NDWI_fn ,swir_fn))
#     comb_img = np.concatenate((read_NDWI_img,read_swir_img), axis=0)
#     comb_img = np.concatenate((read_NDWI_img,read_red_img,read_green_img,read_blue_img, read_swir_img), axis=0)
#     comb_img = np.transpose(comb_img, axes=(1,2,0)) #Channel at the end  ( , ,1) 
    
    annotation_im = Image.open(os.path.join(path_to_write, fn.replace(NDWI_fn,annotation_fn)))
    annotation = np.array(annotation_im)
    
    f = FrameInfo(comb_img, annotation)
    return f

In [42]:
#load testing dataset, use all sequential patchs
frames=[]
for i in range(0,type_num):
    path_to_write=os.path.join(base_dir,'patchesReshape\\test\\type'+str(i))
    all_files = os.listdir(path_to_write)
    all_files_NDWI = [fn for fn in all_files if fn.startswith(NDWI_fn) and fn.endswith(image_type)]#ndwi.png
    print('type{} image number:{}'.format(i,len(all_files_NDWI)))
    for j, fn in enumerate(all_files_NDWI):
        f =readImgs(path_to_write,fn)
        frames.append(f)
random.shuffle(frames)
print('total img count:'+str(len(frames)))
test_patchs = DataGenerator(input_image_channel, patch_size, frames, input_label_channel, augmenter = None).all_sequential_patches(step_size)
print('test patchs number:',len(test_patchs[0]))

type0 image number:14
type1 image number:14
type2 image number:5
type3 image number:11
type4 image number:1
total img count:45
test patchs number: 773


In [43]:
#Evaluate model 

def evaluate_model(model_path, evaluation_report_filename):
    print(model_path, evaluation_report_filename)
    model = load_model(model_path, custom_objects={'dice_loss':dice_loss, 'accuracy':accuracy , 'recall':recall, 'precision':precision,'IoU':IoU}, compile=False)
    model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=[dice_loss, accuracy,recall,precision,IoU])
    
    print('Evaluating model now!')
    ev = model.evaluate(test_patchs[0], test_patchs[1],  verbose=1, use_multiprocessing=False)
    report  = dict(zip(model.metrics_names, ev))
    report['model_path'] =  model_path   
    report['test_frame_dir']= base_dir   
    report['total_patch_count']= len(test_patchs[0])  
    return report

report = evaluate_model(modelToEvaluate, evaluation_report_filename)

D:\sample250\U-Net_13\saved_models\UNet\lakes_20230819-1052_AdaDelta_dice_loss_b1_01_512.h5 D:\sample250\U-Net_13\evaluationreport\evaluation_per_pixel20230819-1507_012.csv
Evaluating model now!
25/25 [==============================] - 16s 581ms/step - loss: 0.2739 - dice_loss: 0.2752 - accuracy: 0.9919 - recall: 0.7265 - precision: 0.7726 - IoU: 0.5877


In [44]:
# Generate the final report
print(report)

tdf = pd.DataFrame(report, index=[0])  
print(tdf.columns)
col_beginning = ['model_path','test_frame_dir', 'total_patch_count', 'accuracy', 'recall','precision','IoU']

col_rest = [x for x in tdf.columns.tolist() if x not in col_beginning]
cols = col_beginning + col_rest
tdf = tdf[cols]
tdf.to_csv(evaluation_report_filename)

{'loss': 0.27391839027404785, 'dice_loss': 0.2752492129802704, 'accuracy': 0.9919134378433228, 'recall': 0.7264832258224487, 'precision': 0.7725878953933716, 'IoU': 0.5876513123512268, 'model_path': 'D:\\sample250\\U-Net_13\\saved_models\\UNet\\lakes_20230819-1052_AdaDelta_dice_loss_b1_01_512.h5', 'test_frame_dir': 'D:\\sample250\\U-Net_13', 'total_patch_count': 773}
Index(['loss', 'dice_loss', 'accuracy', 'recall', 'precision', 'IoU',
       'model_path', 'test_frame_dir', 'total_patch_count'],
      dtype='object')
